In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT REGEXP_EXTRACT(i, '([0-9])', 1) || REGEXP_EXTRACT(REVERSE(i), '([0-9])', 1) i
        FROM (
            SELECT UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    )
    SELECT SUM(CAST(i AS INT)) ans
    FROM t
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│  55477 │
└────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), digits AS (
        SELECT --zero not in valid range
            UNNEST(ARRAY['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']) txt,
            UNNEST(ARRAY['1', '2', '3', '4', '5', '6', '7', '8', '9']) nbr
    ), t AS (
        SELECT
            REGEXP_EXTRACT(i, (
                SELECT '([0-9]|' || STRING_AGG(txt, '|') || ')'
                FROM digits
            ), 1) o1,
            REVERSE(REGEXP_EXTRACT(REVERSE(i), (
                SELECT '([0-9]|' || STRING_AGG(REVERSE(txt), '|') || ')'
                FROM digits
            ), 1)) o2
        FROM (
            SELECT LOWER(UNNEST(STRING_SPLIT(input, '\n'))) i
            FROM input
        )
    ), t_numbers AS (
        SELECT
            (
                SELECT nbr FROM digits WHERE txt = t.o1 OR nbr = t.o1
            ) || (
                SELECT nbr FROM digits WHERE txt = t.o2 OR nbr = t.o2
            ) o
        FROM t
    )
    SELECT SUM(CAST(o AS INT)) ans
    FROM t_numbers
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│  54431 │
└────────┘